In [33]:
# 读入statename文件
import pandas as pd
statename_path = "/home/lijh/knowledge-graph/statename.csv"
statename = pd.read_csv(statename_path)

In [ ]:
generator_path = "/home/lijh/knowledge-graph/Generator.csv"
generator = pd.read_csv(generator_path)

In [47]:
from neo4j import GraphDatabase

# 创建数据库连接
class KnowledgeGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_node(self, label, properties):
        with self.driver.session() as session:
            session.execute_write(self._create_and_return_node, label, properties)

    @staticmethod
    def _create_and_return_node(tx, label, properties):
        query = (
            f"CREATE (n:{label} {{name: $name}}) "
            "RETURN n"
        )
        tx.run(query, name=properties['name'])

    def create_relationship(self, node1, node2, relationship_type):
        with self.driver.session() as session:
            session.execute_write(self._create_and_return_relationship, node1, node2, relationship_type)

    @staticmethod
    def _create_and_return_relationship(tx, node1, node2, relationship_type):
        query = (
            "MATCH (a {name: $node1_name}), (b {name: $node2_name}) "
            f"CREATE (a)-[r:{relationship_type}]->(b) "
            "RETURN type(r)"
        )
        tx.run(query, node1_name=node1, node2_name=node2)

# 使用示例
if __name__ == "__main__":
    # 连接到Neo4j数据库
    kg = KnowledgeGraph("neo4j+s://0887a3e7.databases.neo4j.io", "neo4j", "fZ9mI0LlBpKhp2HE2Oa_ZLy3j0gQQmtgAnWGl8bBNtE")

    # 创建节点
    # kg.create_node("EnergySource", {"name": "Solar"})
    # kg.create_node("EnergySource", {"name": "Wind"})
    # kg.create_node("Location", {"name": "California"})
    # 假定 `kg` 是你的 Neo4j 连接对象
    with kg.driver.session() as session:
    # 在循环外查询现有的县
        existing_counties = set(record['name'] for record in session.run("MATCH (c:County) RETURN c.name AS name"))
    for index, row in generator.iterrows():
        if row['State'] == 'TX':
            county_name = row['County']
            if county_name not in existing_counties:
                # 使用参数化查询创建节点
                session.run("CREATE (c:County {name: $name})", name=county_name)


    # 创建关系
    # kg.create_relationship("Solar", "California", "LOCATED_IN")
    # kg.create_relationship("Wind", "California", "LOCATED_IN")

    # 关闭连接
    kg.close()


In [51]:
test_path = "/home/lijh/knowledge-graph/Multifuel.xlsx"
c = pd.read_excel(test_path)
c.to_csv("/home/lijh/knowledge-graph/Multifuel", index=False)


In [ ]:
# 创建Solar,Wind,Multifuel,Energy_Storage,Other_generator节点
        # 创建Solar_TX中的Solar节点
        for index, row in Solar_TX.iterrows():
            name = f"{row['Plant Name']}_{row['Generator ID']}"
            session.run("CREATE (s:Solar {name: $name, technology: $technology})", 
                name=name, technology=row['Technology'])
        
        # 创建Wind_TX中的Wind节点
        for index, row in Wind_TX.iterrows():
            name = f"{row['Plant Name']}_{row['Generator ID']}"
            session.run("CREATE (w:Wind {name: $name, technology: $technology})", 
                name=name, technology=row['Technology'])
        
        # 创建Multifuel_TX中的Multifuel节点
        for index, row in Multifuel_TX.iterrows():
            name = f"{row['Plant Name']}_{row['Generator ID']}"
            session.run("CREATE (m:Multifuel {name: $name, technology: $technology})", 
                name=name, technology=row['Technology'])
        
        # 创建Energy_storage_TX中的Energy_storage节点
        for index, row in Energy_storage_TX.iterrows():
            name = f"{row['Plant Name']}_{row['Generator ID']}"
            session.run("CREATE (e:Energy_storage {name: $name, technology: $technology})", 
                name=name, technology=row['Technology'])
        
        # 创建Other_generation_TX中的Other_generation节点
        for index, row in Other_generation_TX.iterrows():
            name = f"{row['Plant Name']}_{row['Generator ID']}"
            session.run("CREATE (o:Other_generation {name: $name, technology: $technology})", 
                name=name, technology=row['Technology'])

In [ ]:
import pandas as pd
from neo4j import GraphDatabase
# 读取数据，这里的数据通过dataprocess.ipynb处理后得到
# Generator_TX : 德州所有发电厂数据
# Solar_TX : 德州太阳能发电厂数据
# Wind_TX : 德州风力发电厂数据
# Other_generator_TX : 德州其他类型发电厂数据
# Multifuel_TX : 德州多燃料发电厂数据
# Energy_Storage_TX : 德州所有储能厂数据
Generator_TX = pd.read_csv('/home/lijh/knowledge-graph/TX_data/Generator_TX.csv')
Solar_TX = pd.read_csv('/home/lijh/knowledge-graph/TX_data/Solar_TX.csv')
Wind_TX = pd.read_csv('/home/lijh/knowledge-graph/TX_data/Wind_TX.csv')
Other_generator_TX = pd.read_csv('/home/lijh/knowledge-graph/TX_data/Other_generator_TX.csv')
Multifuel_TX = pd.read_csv('/home/lijh/knowledge-graph/TX_data/Multifuel_TX.csv')
Energy_Storage_TX = pd.read_csv('/home/lijh/knowledge-graph/TX_data/Energy_Storage_TX.csv')
# 这个类是用来连接到Neo4j数据库的，从官方文档中复制过来的
class KnowledgeGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_relationship(self, node1, node2, relationship_type):
        with self.driver.session() as session:
            session.execute_write(self._create_and_return_relationship, node1, node2, relationship_type)

    @staticmethod
    def _create_and_return_relationship(tx, node1, node2, relationship_type):
        query = (
            "MATCH (a {name: $node1_name}), (b:County {name: $node2_name}) "
            f"MERGE (a)-[r:{relationship_type}]->(b) "
            "RETURN type(r)"
        )
        tx.run(query, node1_name=node1, node2_name=node2)
# 以下两个函数是针对此项目自建的函数，用于遍历DataFrame中的每一行，循环调用官方create函数创建节点和关系
# 此函数是创建各种机组节点，这里的节点是发电厂机组，包含了机组名称和技术类型
def create_nodes(session, df, label):
    for index, row in df.iterrows():
        name = f"{row['Plant Name']}_{row['Generator ID']}"
        session.run(
            f"MERGE (n:{label} {{name: $name, technology: $technology}})", 
            name=name, technology=row['Technology']
        )
# 此函数是建立各种机组和县的所在关系，这里的关系是located_in，表示机组位于某个县
def create_relationships(df, kg):
    for index, row in df.iterrows():
        node_name = f"{row['Plant Name']}_{row['Generator ID']}"
        county_name = row['County']
        Utility_name = row['Utility Name']
        kg.create_relationship(node_name, Utility_name, "owned_by")
# 主函数，创建KnowledgeGraph对象，连接到数据库，调用上面的函数，最后关闭连接
if __name__ == "__main__":
    kg = KnowledgeGraph("neo4j+s://0887a3e7.databases.neo4j.io", "neo4j", "fZ9mI0LlBpKhp2HE2Oa_ZLy3j0gQQmtgAnWGl8bBNtE")
    # 创建一个session，用于执行cypher语句，with语句保证了session的自动关闭
    with kg.driver.session() as session:
        # 调用创建机组和电力公司关系的函数
        create_relationships(Solar_TX, kg)
    # 关闭连接
    kg.close()
